# Intelligent Systems | HS2025 | SW12 
Homework - Fine-Tuning a YOLO11 model with images from Kangaroos

## Step 2 - Convert the XML files in folder "kangoroo/annotations" into TXT files.

The Kangaroo images are anotated (with bounding boxes), the details of the bounding boxes are in a special XML format. YOLO needs the bounding box information in a different format, stored in a TXT file.
This Notebook converts the XML files into a TXT file suitable for training a YOLO model.

You don't need to change anything in the Notebook, just run it.

If you get an error that a library is missing, just install the library with the pip command.

When done, you should seee a new folder called "kangaroo/labels/, with one .txt file for each image. For example, the content of the text file "00001.txt" will look like this, describing two bounding boxes, because the images contains two Kangaroos:

```
0 0.687778 0.550157 0.340000 0.542320
0 0.527778 0.561129 0.460000 0.463950
```




In [2]:
import os
import xml.etree.ElementTree as ET
from pathlib import Path

In [3]:
# Get the current folder
current_folder = Path().resolve()

In [5]:
# Paths to annotations and images
annotations_path = "kangaroo/annotations"
images_path =  "kangaroo/images"
labels_path =  "kangaroo/labels"

# Create labels directory if it doesn't exist
os.makedirs(labels_path, exist_ok=True)

# Define class mapping
class_mapping = {"kangaroo": 0}  # Mapping class names to IDs

def convert_annotation(xml_file):
    # Parse the XML file
    tree = ET.parse(xml_file)
    root = tree.getroot()
    
    # Get image size
    size = root.find("size")
    img_width = int(size.find("width").text)
    img_height = int(size.find("height").text)
    
    # Prepare the YOLO label content
    yolo_labels = []
    
    # Iterate over all objects in the XML file
    for obj in root.findall("object"):
        class_name = obj.find("name").text
        if class_name not in class_mapping:
            continue  # Skip classes not in our mapping
        
        class_id = class_mapping[class_name]
        bndbox = obj.find("bndbox")
        xmin = int(bndbox.find("xmin").text)
        ymin = int(bndbox.find("ymin").text)
        xmax = int(bndbox.find("xmax").text)
        ymax = int(bndbox.find("ymax").text)
        
        # Convert to YOLO format
        x_center = ((xmin + xmax) / 2) / img_width
        y_center = ((ymin + ymax) / 2) / img_height
        width = (xmax - xmin) / img_width
        height = (ymax - ymin) / img_height
        yolo_labels.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    
    return yolo_labels

# Convert all XML files in the annotations folder
for xml_file in os.listdir(annotations_path):
    if not xml_file.endswith(".xml"):
        continue
    
    xml_path = os.path.join(annotations_path, xml_file)
    yolo_labels = convert_annotation(xml_path)
    
    # Write YOLO labels to a .txt file
    label_file = os.path.join(labels_path, os.path.splitext(xml_file)[0] + ".txt")
    with open(label_file, "w") as f:
        f.write("\n".join(yolo_labels))

print("Conversion completed. YOLO labels saved in 'labels' folder.")

Conversion completed. YOLO labels saved in 'labels' folder.
